In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as exp


In [ ]:
data = pd.read_csv('BX-Users.csv')
ratings = pd.read_csv('BX-Book-Ratings.csv')


In [ ]:
#---katharismos tou column location gia na meinei mono h xwra---
data.location = data.location.apply(lambda x: x.split(', ')[-1])

# clear people above 110 age
data = data.query("age >= 10 and age <= 110")

#---katharismos twn countries kathws uphrxan mesa mi uparktes xwres.---
unique_countries = data['location'].unique()
#arxikos arithmos xwrwn
print("Xwres prin to katharisma: ", len(unique_countries))

location_freq = data.location.value_counts()
invalid_countries = location_freq[location_freq < 15].index
data = data[~data.location.isin(invalid_countries)]

unique_countries = data['location'].unique()
print("Xwres meta to katharisma: ", len(unique_countries))

In [ ]:
#function gia na plottarei tis meses hlikies kathe xwras
def makePlot(number): 

    for x in range(number):
        country = unique_countries[x]
        age18_25 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 25) & (data[data['location'] == country]['age'] >= 18)]
        age26_35 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 35) & (data[data['location'] == country]['age'] >= 26)]
        age36_45 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 45) & (data[data['location'] == country]['age'] >= 36)]
        age46_55 = data[data['location'] == country]['age'][(data[data['location'] == country]['age'] <= 55) & (data[data['location'] == country]['age'] >= 46)]
        age55above = data[data['location'] == country]['age'][data[data['location'] == country]['age'] >= 56]

        x = ["18-25","26-35","36-45","46-55","55+"]
        y = [len(age18_25.values),len(age26_35.values),len(age36_45.values),len(age46_55.values),len(age55above.values)]

        plt.figure(figsize=(15,6))
        sns.barplot(x=x, y=y, palette="rocket")
        plt.title(country)
        plt.xlabel("Age")
        plt.ylabel("COUNT OF PEOPLE")
        plt.show()  
#makePlot(5)


In [ ]:
#vazoume gia kathe xwra ena monadiko arithmo gia na perasei sto model
for country in unique_countries:
    #print(country, np.where(unique_countries == country)[0][0])
    data['location'] = data['location'].replace(country, np.where(unique_countries == country)[0][0])

#dropparoume tis Nan times gia na mporei na mpei sto kmean model
data = data.dropna()


In [ ]:
X = np.array(data.loc[:,['location', 'age']])


kmeans = KMeans(init="random",n_clusters=3,n_init=10,max_iter=300,random_state=42)

model = kmeans.fit_predict(X)

# to model einai ta labels gia to pou anikei o kathe user
#print(len(model))


plt.scatter(X[:,0], X[:,1])
plt.scatter(kmeans.cluster_centers_[:, 0], 
            kmeans.cluster_centers_[:, 1], 
            s=20,     # Set centroid size
            c='red')  # Set centroid color
plt.show()
 

# enallagh pali se xwres anti gia arithmous gia na proxwrisoume se diaxwrismo
for country in unique_countries:
    data['location'] = data['location'].replace(np.where(unique_countries == country)[0][0], country) 


data = data.reset_index(drop=True)

#eisagoume neo column sta data me to se poio cluster anikei o kahte user
data['cluster'] = model

# edw ksexwrizoyme tous users me vasi to cluster pou tous anatehike
# isws xreiastei reset index
clust_1 = data[data['cluster'] == 0]
clust_2 = data[data['cluster'] == 1]
clust_3 = data[data['cluster'] == 2]

#print(data.head)

zero_ratings = ratings[ratings['rating'] == 0]
zero_ratings = zero_ratings.reset_index(drop=True)
#zero_ratings.head


non_zero_ratings = ratings[ratings['rating'] != 0]
non_zero_ratings = non_zero_ratings.reset_index(drop=True)


In [ ]:
clust_1_users_ratings = pd.merge(clust_1, non_zero_ratings, how='inner', on='uid')
clust_2_users_ratings = pd.merge(clust_2, non_zero_ratings, how='inner', on='uid')
clust_3_users_ratings = pd.merge(clust_3, non_zero_ratings, how='inner', on='uid')

clust_1_users_ratings.head()


In [69]:
zero_ratingss = zero_ratings.iloc[:10]

In [ ]:
books_rat_clust1 = np.empty((0, 2), int)
books_rat_clust2 = np.empty((0, 2), int)
books_rat_clust3 = np.empty((0, 2), int)



for book in range(len(zero_ratings)):
    
    # etsi pairno me tin seira isbn apo ta zero
    x = zero_ratings.iloc[book]['isbn']

    # elegxei an yparxei to isbn sta non zero
    if x in clust_1_users_ratings['isbn'].values and (x in books_rat_clust1) == False:
        # poses fores uparxei auto to isbn sta non zero
        #clust_1_users_ratings['isbn'].value_counts()[x]
        
        # edw vazo se ena neo dataframe oles tis vathmologies twn allwn users gia to biblio
        ongoing_book_rating=clust_1_users_ratings[clust_1_users_ratings['isbn'] == x]
        # vrisko tin mesi timi giauto to vivlio gia tous user pou to exoun vathmologisei
        mean=ongoing_book_rating['rating'].mean()
        # vazoume sto array to isbn kai dipla ton meso oro twn vathmologiwn
        books_rat_clust1=np.append(books_rat_clust1, np.array([[x, mean]]), axis=0)

    if x in clust_2_users_ratings['isbn'].values and (x in books_rat_clust2) == False:
        
        # edw vazo se ena neo dataframe oles tis vathmologies twn allwn users gia to biblio
        ongoing_book_rating=clust_2_users_ratings[clust_2_users_ratings['isbn'] == x]
        # vrisko tin mesi timi giauto to vivlio gia tous user pou to exoun vathmologisei
        mean=ongoing_book_rating['rating'].mean()
        # vazoume sto array to isbn kai dipla ton meso oro twn vathmologiwn
        books_rat_clust2=np.append(books_rat_clust2, np.array([[x, mean]]), axis=0)

    if x in clust_3_users_ratings['isbn'].values and (x in books_rat_clust3) == False:
      
        # edw vazo se ena neo dataframe oles tis vathmologies twn allwn users gia to biblio
        ongoing_book_rating=clust_3_users_ratings[clust_3_users_ratings['isbn'] == x]
        # vrisko tin mesi timi giauto to vivlio gia tous user pou to exoun vathmologisei
        mean=ongoing_book_rating['rating'].mean()
        # vazoume sto array to isbn kai dipla ton meso oro twn vathmologiwn
        books_rat_clust3=np.append(books_rat_clust3, np.array([[x, mean]]), axis=0)        



In [ ]:
print(len(clust_1_users_ratings))
print(len(clust_2_users_ratings))
print(len(clust_3_users_ratings))

x = zero_ratings.iloc[3]['isbn']
print((x in books_rat_clust3) == False)


# KAPOS EPSAXNA NA VRO TA MONADIKA ISBNS KAI EBGAINAN POLLA
# LOGIKA UPARXEI THEMA EKEI KAI PRIN TO TREKSO TIN EPOMENO FORA PREPEI AN TO DIORTHOSO KAPOS

# PREPEI NA FTIAKSO PROTA NA TSEKAREI AN UPARXEI STIN TELIKH LISTA TWN MESO ORON TO VIVLIO KAI META NA TA VAZEI MESA

#ΑΝΑΘΕΩΡΗΣΗ ΓΙΑ ΤΟ ΑΝ ΠΡΕΠΕΙ ΝΑ ΒΓΑΛΩ απο τα ratings ΤΟΥΣ user που δεν υπαρχουν στο user table για να μεινουν λιγοτεροι

# πρεπει επισης να φτιαξω και λιγο το σφαλμα κατα το τσεκ αν υπαρχει πιο πανω βαζοντας μια μεταβλητη....αυατα